# How to Develop Simple Methods for Univariate Forecasting

In [1]:
# one-step naive forecast
def naive_forecast(history, n):
    return history[-n]

In [2]:
# define dataset
data = [10,20,30,40,50,60,70,80,90,100]
print(data)
# test naive forecast
for i in range(1, len(data)+1):
    print(naive_forecast(data,i))


[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
100
90
80
70
60
50
40
30
20
10


In [6]:
from numpy import mean, median
# one-step average forecast
def average_forecast(history: list, config: tuple):
    n, avg_type = config
    # mean of last n values
    if avg_type is 'mean':
        return mean(history[-n:])
    # median of last n values
    return median(history[-n:])

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\pc\AppData\Local\Temp\ipykernel_4892\2103709765.py:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if avg_type is 'mean':


In [7]:
# define dataset
data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
print(data)
# test naive forecast
for i in range(i, len(data)+1):
    print(average_forecast(data, (i, 'mean')))

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
55.0
